# Introduction

Welcome to this repository. We will be walking you to a series of notebooks in which you will understand how RAG works (Retrieval Augmented Generation, a technique that combines the power of search and generation of AI to answer user queries). We will work with different sources (Azure AI Search, Files, SQL Server, Websites, APIs, etc) and at the end of the notebooks you will understand why the magic happens with the combination of:

1) Multi-Agents: Agents talking to each other
2) Azure OpenAI models
3) Very detailed prompts

But we need to start from the basics, so let's begin with Azure AI Search and how it works:

# Load and Enrich multiple file types Azure AI Search

In this Jupyter Notebook, we create and run enrichment steps to unlock searchable content in the specified Azure blob. It performs operations over mixed content in Azure Storage, such as images and application files, using a skillset that analyzes and extracts text information that becomes searchable in Azure Cognitive Search. 
The reference sample can be found at [Tutorial: Use Python and AI to generate searchable content from Azure blobs](https://docs.microsoft.com/azure/search/cognitive-search-tutorial-blob-python).

In this demo we are going to be using a private (so we can mimic a private data lake scenario) Blob Storage container that has ~9.8k Computer Science publication PDFs from the Arxiv dataset.
https://www.kaggle.com/datasets/Cornell-University/arxiv

If you want to explore the dataset, go [HERE](https://console.cloud.google.com/storage/browser/arxiv-dataset/arxiv/cs/pdf?pageState=(%22StorageObjectListTable%22:(%22f%22:%22%255B%255D%22))&prefix=&forceOnObjectsSortingFiltering=false)<br>
Note: This dataset has been copy to a public azure blob container for this demo

Although only  PDF files are used here, this can be done at a much larger scale and Azure Cognitive Search supports a range of other file formats including: Microsoft Office (DOCX/DOC, XSLX/XLS, PPTX/PPT, MSG), HTML, XML, ZIP, and plain text files (including JSON).
Azure Search support the following sources: [Data Sources Gallery](https://learn.microsoft.com/EN-US/AZURE/search/search-data-sources-gallery)

This notebook creates the following objects on your search service:

+ data source
+ search index
+ skillset
+ indexer

This notebook calls the [Search REST APIs](https://docs.microsoft.com/rest/api/searchservice/), but you can also use the Azure.Search.Documents client library in the Azure SDK for Python to perform the same steps. See this [Python quickstart](https://docs.microsoft.com/azure/search/search-get-started-python) for details.

To run this notebook, you should have already created the Azure services on README. Once you've done this, you can run all cells, but the query won't return results until the indexer is finished and the search index is loaded. 

We recommend running each step and making sure it completes before moving on.

![cog-search](./images/Cog-Search-Enrich.png)

In [1]:
import os
import json
import requests
from dotenv import load_dotenv
load_dotenv("credentials.env")

# Name of the container in your Blob Storage Datasource ( in credentials.env)
BLOB_CONTAINER_NAME = "arxivcs"

In [2]:
# Define the names for the data source, skillset, index and indexer
datasource_name = "cogsrch-datasource-files"
index_name = "cogsrch-index-files"
skillset_name = "cogsrch-skillset-files"
indexer_name = "cogsrch-indexer-files"

In [3]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

## Create Data Source (Blob container with the Arxiv CS pdfs)

In [4]:
# The following code sends the json paylod to Azure Search engine to create the Datasource

datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities.",
    "type": "azureblob",
    "credentials": {
        "connectionString": os.environ['BLOB_CONNECTION_STRING']
    },
    "dataDeletionDetectionPolicy" : {
        "@odata.type" :"#Microsoft.Azure.Search.NativeBlobSoftDeleteDeletionDetectionPolicy" # this makes sure that if the item is deleted from the source, it will be deleted from the index
    },
    "container": {
        "name": BLOB_CONTAINER_NAME
    }
}
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


- 201 - Successfully created
- 204 - Succesfully overwritten
- 40X - Authentication Error

For information on Change and Delete file detection please see [HERE](https://learn.microsoft.com/en-us/azure/search/search-howto-index-changed-deleted-blobs?tabs=rest-api)

In [5]:
# If you have a 403 code, probably you have a wrong endpoint or key, you can debug by uncomment this
# r.text

## Create Index

In Azure AI Search, a search index is your searchable content, available to the search engine for indexing, full text search, vector search, hybrid search, and filtered queries. An index is defined by a schema and saved to the search service, with data import following as a second step. This content exists within your search service, apart from your primary data stores, which is necessary for the millisecond response times expected in modern search applications. Except for indexer-driven indexing scenarios, the search service never connects to or queries your source data.

Reference:

https://learn.microsoft.com/en-us/azure/search/search-what-is-an-index

Notice below how we are creating a vector store. In Azure AI Search, a vector store has an index schema that defines vector and nonvector fields, a vector configuration for algorithms that create the embedding space, and settings on vector field definitions that are used in query requests. 

We are also setting a semantic ranking over a result set, promoting the most semantically relevant results to the top of the stack. You can also get semantic captions, with highlights over the most relevant terms and phrases, and semantic answers.

In [6]:
# Create an index
# Queries operate over the searchable fields and filterable fields in the index
index_payload = {
    "name": index_name,
    "vectorSearch": {
        "algorithms": [
            {
                "name": "myalgo",
                "kind": "hnsw"
            }
        ],
        "vectorizers": [
            {
                "name": "openai",
                "kind": "azureOpenAI",
                "azureOpenAIParameters":
                {
                    "resourceUri" : os.environ['AZURE_OPENAI_ENDPOINT'],
                    "apiKey" : os.environ['AZURE_OPENAI_API_KEY'],
                    "deploymentId" : os.environ['EMBEDDING_DEPLOYMENT_NAME']
                }
            }
        ],
        "profiles": [
            {
                "name": "myprofile",
                "algorithm": "myalgo",
                "vectorizer":"openai"
            }
        ]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "title"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "chunk"
                        }
                    ],
                    "prioritizedKeywordsFields": []
                }
            }
        ]
    },
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "analyzer": "keyword", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false","facetable": "false"},
        {"name": "ParentKey", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "true", "sortable": "false"},
        {"name": "title", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "true", "sortable": "false"},
        {"name": "name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},   
        {"name": "chunk","type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {
            "name": "chunkVector",
            "type": "Collection(Edm.Single)",
            "dimensions": 1536, # IMPORTANT: Make sure these dimmensions match your embedding model name
            "vectorSearchProfile": "myprofile",
            "searchable": "true",
            "retrievable": "true",
            "filterable": "false",
            "sortable": "false",
            "facetable": "false"
        }
    ]
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)


201
True


In [7]:
# r.text

#### Semantic Search capabilities
As you can see above in the index payload, there is a `semantic configuration`. What is that?

Semantic ranker is a collection of query-related capabilities that improve the quality of an initial BM25-ranked or RRF-ranked search result for text-based queries. When you enable it on your search service, semantic ranking extends the query execution pipeline in two ways:

    First, it adds secondary ranking over an initial result set that was scored using BM25 or RRF. This secondary ranking uses multi-lingual, deep learning models adapted from Microsoft Bing to promote the most semantically relevant results.

    Second, it extracts and returns captions and answers in the response, which you can render on a search page to improve the user's search experience.


For deeper explanation and limitations see [HERE](https://learn.microsoft.com/en-us/azure/search/semantic-ranking)

## Create Skillset - OCR, Text Splitter, AzureOpenAIEmbeddingSkill

We need to create now the skillset. This is a set of steps in which we use AI Services to enrich the documents by extracting information, applying OCR, splitting, and embedding chunks, among other skills.

https://learn.microsoft.com/en-us/azure/search/cognitive-search-working-with-skillsets

https://learn.microsoft.com/en-us/azure/search/cognitive-search-predefined-skills


Notice below that we are using IndexProjections. By default, one document processed within a skillset maps to a single document in the search index. This means that if you perform chunking of an input text and then perform enrichments on each chunk, the result in the index when mapped via outputFieldMappings is an array of the generated enrichments. **With index projections, you define a context at which to map each chunk of enriched data to its own search document**. This allows you to apply a one-to-many mapping of a document's enriched data to the search index.
    
The parameter: `"projectionMode": "skipIndexingParentDocuments"` allows us to skip the indexing of the parent documents, and keep only the index with the chunks and its vectors.

### Content Lifecycle
If the indexer data source supports change tracking and deletion detection, the indexing process can synchronize the primary (parend documents) and secondary indexes (chunks) to pick up those changes.

Each time you run the indexer and skillset, the index projections are updated if the skillset or underlying source data has changed. Any changes picked up by the indexer are propagated through the enrichment process to the projections in the index, ensuring that your projected data is a current representation of content in the originating data source. This will save you weeks of programming and a lot of headaches trying to keep the content in sync.

In [8]:
# Create a skillset
skillset_payload = {
    "name": skillset_name,
    "description": "e2e Skillset for RAG - Files",
    "skills":
    [
        {
            "@odata.type": "#Microsoft.Skills.Vision.OcrSkill",
            "description": "Extract text (plain and structured) from image.",
            "context": "/document/normalized_images/*",
            "defaultLanguageCode": "en",
            "detectOrientation": True,
            "inputs": [
                {
                  "name": "image",
                  "source": "/document/normalized_images/*"
                }
            ],
                "outputs": [
                {
                  "name": "text",
                  "targetName" : "images_text"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.MergeSkill",
            "description": "Create merged_text, which includes all the textual representation of each image inserted at the right location in the content field. This is useful for PDF and other file formats that supported embedded images.",
            "context": "/document",
            "insertPreTag": " ",
            "insertPostTag": " ",
            "inputs": [
                {
                  "name":"text", "source": "/document/content"
                },
                {
                  "name": "itemsToInsert", "source": "/document/normalized_images/*/images_text"
                },
                {
                  "name":"offsets", "source": "/document/normalized_images/*/contentOffset"
                }
            ],
            "outputs": [
                {
                  "name": "mergedText", 
                  "targetName" : "merged_text"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
            "context": "/document",
            "textSplitMode": "pages",  # although it says "pages" it actally means chunks, not actual pages
            "maximumPageLength": 5000, # 5000 characters is default and a good choice
            "pageOverlapLength": 750,  # 15% overlap among chunks
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/merged_text"
                }
            ],
            "outputs": [
                {
                    "name": "textItems",
                    "targetName": "chunks"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.AzureOpenAIEmbeddingSkill",
            "description": "Azure OpenAI Embedding Skill",
            "context": "/document/chunks/*",
            "resourceUri": os.environ['AZURE_OPENAI_ENDPOINT'],
            "apiKey": os.environ['AZURE_OPENAI_API_KEY'],
            "deploymentId": os.environ['EMBEDDING_DEPLOYMENT_NAME'],
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/chunks/*"
                }
            ],
            "outputs": [
                {
                    "name": "embedding",
                    "targetName": "vector"
                }
            ]
        }
    ],
    "indexProjections": {
        "selectors": [
            {
                "targetIndexName": index_name,
                "parentKeyFieldName": "ParentKey",
                "sourceContext": "/document/chunks/*",
                "mappings": [
                    {
                        "name": "title",
                        "source": "/document/title"
                    },
                    {
                        "name": "name",
                        "source": "/document/name"
                    },
                    {
                        "name": "location",
                        "source": "/document/location"
                    },
                    {
                        "name": "chunk",
                        "source": "/document/chunks/*"
                    },
                    {
                        "name": "chunkVector",
                        "source": "/document/chunks/*/vector"
                    }
                ]
            }
        ],
        "parameters": {
            "projectionMode": "skipIndexingParentDocuments"
        }
    },
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": os.environ['COG_SERVICES_NAME'],
        "key": os.environ['COG_SERVICES_KEY']
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


In [9]:
# print(r.text)

## Create and Run the Indexer - (runs the pipeline)

The three components you have created thus far (data source, skillset, index) are inputs to an indexer. Creating the indexer on Azure Cognitive Search is the event that puts the entire pipeline into motion.

In [10]:
# Create an indexer
indexer_payload = {
    "name": indexer_name,
    "dataSourceName": datasource_name,
    "targetIndexName": index_name,
    "skillsetName": skillset_name,
    "schedule" : { "interval" : "PT30M"}, # How often do you want to check for new content in the data source
    "fieldMappings": [
        {
          "sourceFieldName" : "metadata_title",
          "targetFieldName" : "title"
        },
        {
          "sourceFieldName" : "metadata_storage_name",
          "targetFieldName" : "name"
        },
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "location"
        }
    ],
    "outputFieldMappings":[],
    "parameters":
    {
        "maxFailedItems": -1,
        "maxFailedItemsPerBatch": -1,
        "configuration":
        {
            "dataToExtract": "contentAndMetadata",
            "imageAction": "generateNormalizedImages"
        }
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


In [11]:
# Uncomment if you find an error
# r.text

'{"@odata.context":"https://cog-search-zf4fwhz3gdn64.search.windows.net/$metadata#indexers/$entity","@odata.etag":"\\"0x8DC7052637CA2AC\\"","name":"cogsrch-indexer-files-2","description":null,"dataSourceName":"cogsrch-datasource-files-2","skillsetName":"cogsrch-skillset-files-2","targetIndexName":"cogsrch-index-files-2","disabled":null,"schedule":{"interval":"PT30M","startTime":"2024-05-09T18:03:38.4035486Z"},"parameters":{"batchSize":null,"maxFailedItems":-1,"maxFailedItemsPerBatch":-1,"base64EncodeKeys":null,"configuration":{"dataToExtract":"contentAndMetadata","imageAction":"generateNormalizedImages"}},"fieldMappings":[{"sourceFieldName":"metadata_title","targetFieldName":"title","mappingFunction":null},{"sourceFieldName":"metadata_storage_name","targetFieldName":"name","mappingFunction":null},{"sourceFieldName":"metadata_storage_path","targetFieldName":"location","mappingFunction":null}],"outputFieldMappings":[],"cache":null,"encryptionKey":null}'

Note: If you get a 400 unauthorize error, make sure that you are using the Azure Search MANAGEMENT KEY, not the QUERY key

In [14]:
# Optionally, get indexer status to confirm that it's running
try:
    r = requests.get(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name +
                     "/status", headers=headers, params=params)
    # pprint(json.dumps(r.json(), indent=1))
    print(r.status_code)
    print("Status:",r.json().get('lastResult').get('status'))
    print("Items Processed:",r.json().get('lastResult').get('itemsProcessed'))
    print(r.ok)
    
except Exception as e:
    print("Wait a few seconds until the process starts and run this cell again.")

200
Status: inProgress
Items Processed: 40
True


**When the indexer finishes running we will have all 9.8k documents indexed in your Search Engine!.**

# References

- https://learn.microsoft.com/en-us/azure/search/cognitive-search-tutorial-blob
- https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/search
- https://learn.microsoft.com/en-us/azure/search/search-get-started-vector

# NEXT
In the next notebook 02, we will implement another type of indexing call One-to-Many, in which a single CSV or JSON file can be converted into multiple individual searchable documents in Azure Search. 